# Exchange Trade Protocol

* Introduction to [Modern Portfolio Theory](https://www.quantconnect.com/tutorials/introduction-to-financial-python/modern-portfolio-theory)
* [Practical Portfolio Optimisation](https://pythoninvest.com/long-read/practical-portfolio-optimisation)

In [162]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('default')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Load marketcap data

In [163]:
import sys
sys.path.insert(1, '../lib')
import dataETL as etl
import pandas as pd

if debug:
    import importlib
    etl = importlib.reload(etl)

''' Initialize the dataETL class '''
path = "../data/market_cap_2021-01-01_2022-06-01/"
cls_etl = etl.ExtractLoadTransform(dataPath=path)
''' Get list of data file names '''
_l_fnames = cls_etl.get_file_list()
print("Retrieved %d files in dir: %s " % (len(_l_fnames),path))
''' Load data into dataframe '''
data_df = cls_etl.load_data(_l_fnames)
print("Loaded %d rows %s" % (data_df.shape[0],str(data_df.columns)))
''' Transform data with coin ids in columns '''
market_df = cls_etl.transfrom_data(data_df)
print(market_df.info())
print("Data load and transformation complete!")

Retrieved 63 files in dir: ../data/market_cap_2021-01-01_2022-06-01/ 
Loaded 3637 rows Index(['Date', 'ID', 'Symbol', 'market_cap'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 524 entries, 0 to 523
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      524 non-null    datetime64[ns]
 1   bitcoin   524 non-null    float64       
 2   cardano   524 non-null    float64       
 3   ethereum  524 non-null    float64       
 4   litecoin  524 non-null    float64       
 5   ripple    524 non-null    float64       
 6   solana    524 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 32.8 KB
None
Data load and transformation complete!


## Filtered by date range

In [164]:
import datetime

start_dt = datetime.datetime(2021,2,1)
end_dt = datetime.datetime(2022,6,30)

''' >= start-date '''
if isinstance(start_dt, datetime.date):
    mask = (market_df["Date"] >= start_dt)
    market_df = market_df[mask]
''' <= end-date '''
if isinstance(end_dt, datetime.date):
    mask = (market_df["Date"] <= end_dt)
    market_df = market_df[mask]
market_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493 entries, 31 to 523
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      493 non-null    datetime64[ns]
 1   bitcoin   493 non-null    float64       
 2   cardano   493 non-null    float64       
 3   ethereum  493 non-null    float64       
 4   litecoin  493 non-null    float64       
 5   ripple    493 non-null    float64       
 6   solana    493 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 30.8 KB


In [165]:
import sys
sys.path.insert(1, '../lib')
import assetETP as etp

if debug:
    import importlib
    etp = importlib.reload(etp)

data_name = "coindecks"
cls_etp = etp.ExchangeTradeProtocol(name=data_name)
print(dir(cls_etp))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'days_offset', 'get_simple_returns', 'get_topN_assets', 'get_value_index', 'name', 'p_val', 'rolling_corr', 'window_length']


In [166]:
_l_topNassets = cls_etp.get_topN_assets(data_df, N=3)
print(_l_topNassets)

          Date        ID                 Value
0   2021-05-01   bitcoin  1082721727348.043213
0   2021-05-01  ethereum   321155845972.335144
0   2021-05-01    ripple    73298828829.094406
0   2021-05-02   bitcoin  1080897634123.293579
0   2021-05-02  ethereum   340730211563.778931
..         ...       ...                   ...
0   2021-08-30  ethereum   378786220548.799622
0   2021-08-30   cardano    91897143051.431595
0   2021-08-31   bitcoin   885729258052.631592
0   2021-08-31  ethereum   378977088509.631714
0   2021-08-31   cardano    87653684618.567917

[1572 rows x 3 columns]


## Value Index

In [49]:
import plotly.express as px

index_df = cls_etp.get_value_index(market_df)

_min_date = (index_df["Date"].min()).date()
_max_date = (index_df["Date"].max()).date()
_title = "Asset class value index "+str(_min_date)+" to "+str(_max_date)
fig = px.line(index_df, x="Date", y=index_df.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

/home/gnewy/.local/lib/python3.8/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



## Simple returns

In [71]:
simple_returns_df = cls_etp.get_simple_returns(market_df)

_title = "Expected Returns from "+str(_min_date)+" to "+str(_max_date)
fig = px.line(simple_returns_df, x="Date", y=simple_returns_df.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

/home/gnewy/.local/lib/python3.8/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [140]:
_data_df = data_df.copy()
_l_dates = _data_df['Date'].unique()
print(_data_df.info())
N=3
_l_topNassets = []
for date in _l_dates:
    ''' get assets and sort by market cap '''
    assets = _data_df.loc[_data_df['Date'] == date]
    assets = assets.sort_values(by='market_cap',axis=0, ascending = False)
    topN = assets.head(N)
    _l_assetsID = []
    _l_marketCap = []
    for row in assets.head(N).iterrows():
        _l_assetsID.append(row[1][1])
        _l_marketCap.append(row[1][3])
    asset_dict = {date : { 'assets' : _l_assetsID, 'marketCap' : _l_marketCap}}
    _l_topNassets.append(asset_dict)
print(_l_topNassets)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3637 entries, 0 to 60
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        3637 non-null   object
 1   ID          3637 non-null   object
 2   Symbol      3637 non-null   object
 3   market_cap  3637 non-null   object
dtypes: object(4)
memory usage: 142.1+ KB
None
[{'2021-05-01': {'assets': ['bitcoin', 'ethereum', 'ripple'], 'marketCap': [1082721727348.0432, 321155845972.33514, 73298828829.0944]}}, {'2021-05-02': {'assets': ['bitcoin', 'ethereum', 'ripple'], 'marketCap': [1080897634123.2936, 340730211563.77893, 76025679530.23318]}}, {'2021-05-03': {'assets': ['bitcoin', 'ethereum', 'ripple'], 'marketCap': [1057850321948.5464, 341953594507.36664, 71701291962.31635]}}, {'2021-05-04': {'assets': ['bitcoin', 'ethereum', 'ripple'], 'marketCap': [1069571255195.5188, 398258027094.341, 71679793845.11757]}}, {'2021-05-05': {'assets': ['bitcoin', 'ethereum', 'ripple'], 'm